# Market Data Analysis by FinMAS

This notebook can be used to run an analysis by Market Data crew instead of using the UI / panel app.

In [ ]:
# Convenience function to change the working directory to the root of the project if necessary
import os
from pathlib import Path

if not Path("finmas").exists():
    os.chdir("..")
print(f"Current working directory: {Path.cwd()}")

In [ ]:
%%capture
import time

from IPython.display import Markdown, display

from finmas.constants import defaults
from finmas.crews import MarketDataCrew
from finmas.crews.utils import CrewRunMetrics, get_yaml_config_as_markdown, save_crew_output
from finmas.utils.common import format_time_spent

To be able to run this notebook make sure the necessary API_KEY for the LLM is set in the `.env` file
or set in the environment variables manually

| provider | API_KEY |
| --- | --- |
| groq | GROQ_API_KEY |
| openai | OPENAI_API_KEY |

## Configuration

List of Groq models: https://console.groq.com/settings/limits

In [ ]:
TICKER = "META"
PRICE_END_DATE = defaults["price_end_date"]

# LLM_PROVIDER = "groq"
# LLM_MODEL = "llama3-8b-8192"

# Change the model here if needed
LLM_PROVIDER = "openai"
LLM_MODEL = "gpt-4o"
# LLM_MODEL = "gpt-4o-mini"

# LLM configuration
LLM_TEMPERATURE = 0.0
LLM_MAX_TOKENS = 1024

CREW_NAME = "market_data"

## Crew Configuration

In [ ]:
config_path = Path.cwd() / f"finmas/crews/{CREW_NAME}/config"

### Agents

In [ ]:
display(Markdown(get_yaml_config_as_markdown(config_path, "agents", inputs={"ticker": TICKER})))

### Fundamental Analyst

- **Role**: Fundamental Analyst
- **Goal**: Analyze META's fundamental data to evaluate the company's profitability and growth potential.
- **Backstory**: You are an expert in fundamental analysis of stocks and have a strong understanding of key financial metrics such as revenue growth, earnings per share, and net profit margin.

### Technical Analyst

- **Role**: Technical Analyst
- **Goal**: Analyze META's historical price data to identify trends and patterns that can help predict future price movements.
- **Backstory**: You are an expert in technical indicators for stock prices, and use them to analyze the trend, momentum and volatility of stocks.

### Stock Advisor

- **Role**: Stock Advisor
- **Goal**: Provide investment recommendations to whether buy, sell, or hold META based on fundamental and technical analysis.
- **Backstory**: You are a world class stock picker and provide advice to clients based on fundamental and technical analysis.



### Tasks Definition

In [ ]:
display(Markdown(get_yaml_config_as_markdown(config_path, "tasks", inputs={"ticker": TICKER})))

### Fundamental Analysis

- **Description**: Analyze META's fundamental data to evaluate the company's profitability and growth potential. Use the provided tool to analyze total revenue, net income, earnings per share, net profit margin, and possibly other key financial metrics. Use available Trailing Twelve Months (TTM) data in your analysis if necessary.
- **Expected Output**: The final answer should be a summary of the company's financial health and growth prospects based on the data available.
- **Agent**: fundamental_analyst

### Technical Analysis

- **Description**: Analyze META's historical price data to predict future price movements. Use the provided tool to analyze price trends, momentum, and volatility. For momentum use the RSI indicator, and for volatility use the Bollinger Bands percentage indicator. Use the available historical price data in the tool to analyze the stock's price movements.
- **Expected Output**: The final answer should be a summary of the company's price trends and potential future price movements based on the data available.
- **Agent**: technical_analyst

### Stock Advisor Task

- **Description**: Analyze META's fundamental and technical data to provide a recommendation on whether to buy, sell, or hold the stock. Use the provided input to analyze both fundamental and technical data for a comprehensive view of the stock's investment potential.
- **Expected Output**: The final answer should be a recommendation (buy, sell, or hold) based on the analysis of the company's profitability, historical fundamental data and technical indicators. The recommendation should be supported by the data available, and should be clear and concise. Highlight the latest fundamental and technical data that support your recommendation.
- **Agent**: stock_advisor



## Fundamentals Tool

To demonstrate the tool that the fundamental analyst agent uses, we show the following example output.

This tool extracts the latest fundamental data using Alpha Vantage for income statement and balance sheet, and Yahoo Finance for price data.  
The tool condenses the data into 2 tables formatted as Markdown to make it easier for the LLM agent to interpret the data.

In [ ]:
from finmas.data.market.fundamentals import StockFundamentalsTool

fundamentals_tool = StockFundamentalsTool()

display(Markdown(fundamentals_tool.run(TICKER)))

Using Tool: Stock Fundamentals Tool


## META - Fundamentals

The date of the latest quarter is: 2024-09-30

This table shows some essential fundamental data for the given stock ticker over the last 8 quarters. TTM means Trailing Twelve Months.

|       Date |   Total Revenue |     Net Income |   Net Profit Margin (%) |   Stock Price |   Basic Quarterly Earnings Per Share |   Debt to Equity |   Basic TTM Price to Earnings Ratio |   Price to Sales Ratio TTM |
|------------|-----------------|----------------|-------------------------|---------------|--------------------------------------|------------------|-------------------------------------|----------------------------|
| 2022-12-31 |  31,920,000,000 |  4,653,000,000 |                   14.58 |        119.98 |                                 1.76 |             0.48 |                               13.93 |                       2.72 |
| 2023-03-31 |  28,591,000,000 |  5,709,000,000 |                   19.97 |        211.30 |                                 2.21 |             0.48 |                               26.15 |                       4.67 |
| 2023-06-30 |  31,884,000,000 |  7,788,000,000 |                   24.43 |        286.12 |                                 3.03 |             0.54 |                               33.11 |                       6.12 |
| 2023-09-30 |  34,009,000,000 | 11,583,000,000 |                   34.06 |        299.31 |                                 4.50 |             0.51 |                               26.03 |                       6.10 |
| 2023-12-31 |  39,605,000,000 | 14,017,000,000 |                   35.39 |        352.90 |                                 5.46 |             0.50 |                               23.22 |                       6.75 |
| 2024-03-31 |  36,455,000,000 | 12,369,000,000 |                   33.93 |        484.64 |                                 4.86 |             0.49 |                               27.14 |                       8.69 |
| 2024-06-30 |  39,071,000,000 | 13,465,000,000 |                   34.46 |        503.74 |                                 5.31 |             0.47 |                               25.02 |                       8.56 |
| 2024-09-30 |  40,589,000,000 | 15,688,000,000 |                   38.65 |        572.44 |                                 6.20 |             0.56 |                               26.21 |                       9.30 |

### Year over Year Growth for Trailing Twelve Months

This table shows the year over year growth rates and the trailing twelve months for total revenue, net income, net profit margin, and basic EPS for the given stock ticker over the last 8 quarters.

|       Date |   Total Revenue TTM |   Total Revenue TTM YoY (%) |   Net Income TTM |   Net Income TTM YoY (%) |   Net Profit Margin TTM (%) |   Net Profit Margin TTM YoY (%) |   Basic TTM Earnings Per Share |   Basic TTM Earnings Per Share YoY (%) |
|------------|---------------------|-----------------------------|------------------|--------------------------|-----------------------------|---------------------------------|--------------------------------|----------------------------------------|
| 2022-12-31 |     116,352,000,000 |                       -1.21 |   23,200,000,000 |                   -41.07 |                       19.94 |                          -40.35 |                           8.61 |                                 -38.43 |
| 2023-03-31 |     117,039,000,000 |                       -2.07 |   21,444,000,000 |                   -42.57 |                       18.32 |                          -41.36 |                           8.08 |                                 -39.67 |
| 2023-06-30 |     120,105,000,000 |                        0.71 |   22,545,000,000 |                   -32.96 |                       18.77 |                          -33.44 |                           8.64 |                                 -29.17 |
| 2023-09-30 |     126,404,000,000 |                        7.16 |   29,733,000,000 |                     3.12 |                       23.52 |                           -3.76 |                          11.50 |                                   8.77 |
| 2023-12-31 |     134,089,000,000 |                       15.24 |   39,097,000,000 |                    68.52 |                       29.16 |                           46.23 |                          15.20 |                                  76.46 |
| 2024-03-31 |     141,953,000,000 |                       21.29 |   45,757,000,000 |                   113.38 |                       32.23 |                           75.93 |                          17.85 |                                 120.92 |
| 2024-06-30 |     149,140,000,000 |                       24.17 |   51,434,000,000 |                   128.14 |                       34.49 |                           83.72 |                          20.14 |                                 133.01 |
| 2024-09-30 |     155,720,000,000 |                       23.19 |   55,539,000,000 |                    86.79 |                       35.67 |                           51.63 |                          21.84 |                                  89.94 |

## Technical Analysis Tool

To demonstrate the tool that the technical analyst agent uses, we show the following example output.\
The tool extracts the latest price data using Yahoo Finance and resamples the data to a weekly timeframe.\
The technical indicators 20 week moving average, 50 week moving average are used for indicating the trend of the stock.\
The momentum indicator RSI14 and the volatility indicator Bollinger Band percentage are used\
to indicate the strength of the momentum and the volatility of the stock.

In [ ]:
from finmas.data.market.technical_analysis import TechnicalAnalysisTool

ta_tool = TechnicalAnalysisTool(end_date=PRICE_END_DATE)

display(Markdown(ta_tool.run(TICKER)))

Using Tool: Technical Analysis Tool


## META - Technical Indicators

Date of the latest technical analysis data is: 2024-11-08

This table shows some technical indicators for the given stock ticker over the last 8 weeks. SMA = Simple Moving Average represent the trend. RSI = Relative Strength Index and Bollinger Band represent the momentum.

When RSI is above 70, the stock is considered overbought. When RSI is below 30, the stock is considered oversold.

When the Bollinger Band Percentage is above 100, the stock is considered overbought. When the Bollinger Band Percentage is below 0, the stock is considered oversold.

|       Date |   Close price |   SMA 50 week |   SMA 20 week |   SMA Trend |   RSI 14 |   Bollinger Band Percentage % |
|------------|---------------|---------------|---------------|-------------|----------|-------------------------------|
| 2024-09-20 |        561.35 |        444.82 |        501.40 |          Up |    64.34 |                        108.24 |
| 2024-09-27 |        567.36 |        449.89 |        506.01 |          Up |    65.19 |                        103.42 |
| 2024-10-04 |        595.94 |        455.66 |        512.25 |          Up |    68.95 |                        112.28 |
| 2024-10-11 |        589.95 |        461.54 |        517.89 |          Up |    67.31 |                         99.26 |
| 2024-10-18 |        576.47 |        466.80 |        523.41 |          Up |    63.63 |                         86.19 |
| 2024-10-25 |        573.25 |        471.71 |        527.48 |          Up |    62.75 |                         80.56 |
| 2024-11-01 |        567.16 |        476.37 |        530.65 |          Up |    61.03 |                         74.03 |
| 2024-11-08 |        589.34 |        481.41 |        535.40 |          Up |    64.82 |                         84.52 |

## Create Crew

This section creates the crew with the configuration of agents and tasks defined above.
The crew will use the output from the tools shown above to make a decision on the stock.

In [ ]:
%%capture
start = time.time()
crew = MarketDataCrew(
    ticker=TICKER,
    llm_provider=LLM_PROVIDER,
    llm_model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    max_tokens=LLM_MAX_TOKENS,
    price_end_date=PRICE_END_DATE,
)

## Run the analysis

In [ ]:
%%capture
inputs = {"ticker": TICKER}  # "form": filing.form}
output = crew.crew().kickoff(inputs=inputs)
time_spent = time.time() - start
display(f"Time spent: {format_time_spent(time_spent)}")

## Save the results

In [ ]:
crew_run_metrics = CrewRunMetrics(
    config=crew.config, token_usage=output.token_usage, time_spent=time_spent
)
file_path = save_crew_output(crew_run_metrics, output.raw)
display(f"Output saved to: {file_path}")

'Output saved to: output/market_data/META_market_data_analysis_20241118_124351.md'

## Display the results

In [ ]:
crew_output = file_path.read_text(encoding="utf-8")
display(Markdown(crew_output))

# Crew output:

Based on the comprehensive analysis of META's fundamental and technical data, I recommend a "Buy" for META stock. Here's the rationale behind this recommendation:

**Fundamental Analysis:**

1. **Revenue and Profitability Growth:** META has demonstrated strong financial health with a consistent upward trend in total revenue, which reached $40.589 billion in the latest quarter. The net income has also seen substantial growth, reaching $15.688 billion, indicating a significant improvement in profitability. The net profit margin has improved to 38.65%, showcasing enhanced operational efficiency.

2. **Earnings Per Share (EPS):** The EPS has increased significantly from $1.76 in December 2022 to $6.20 in September 2024, reflecting strong earnings growth.

3. **Growth Potential:** META's revenue growth of 23.19% and net income growth of 86.79% year-over-year highlight its robust growth potential. The EPS growth of 89.94% further underscores the company's potential for future profitability.

4. **Valuation Metrics:** While the price to earnings and price to sales ratios have fluctuated, the overall financial metrics suggest that META is a potentially attractive investment opportunity.

**Technical Analysis:**

1. **Price Trends:** META's stock has shown a strong upward trend, with increasing Simple Moving Averages (SMA) for both 50-week and 20-week periods. This indicates a bullish trend in the stock's price.

2. **Momentum:** The Relative Strength Index (RSI) is at 64.82, below the overbought threshold of 70, suggesting there is still room for upward movement without immediate risk of a reversal.

3. **Volatility:** The Bollinger Band Percentage is at 84.52%, below the overbought level of 100%, indicating moderate volatility and stable price movement.

**Conclusion:**

Both the fundamental and technical analyses support a positive outlook for META. The strong financial health, significant growth potential, and favorable technical indicators suggest that META is well-positioned for continued growth. Therefore, I recommend buying META stock, as it presents a compelling investment opportunity with potential for further appreciation.

## Crew Run Metrics

Total tokens: 6686 Prompt tokens: 5304  
Successful Requests: 5  
Estimated LLM Model cost for total tokens: $0.04725  

Time spent: 48s
## Configuration:  

Crew Name: market_data  
Ticker: META  
LLM: openai / gpt-4o  
Temperature: 0.0 Max tokens: 1024  
Agent Configuration:  
Max iterations: 10 Max requests per minute: 30  

## Agents

### Fundamental Analyst

- **Role**: Fundamental Analyst
- **Goal**: Analyze {ticker}'s fundamental data to evaluate the company's profitability and growth potential.
- **Backstory**: You are an expert in fundamental analysis of stocks and have a strong understanding of key financial metrics such as revenue growth, earnings per share, and net profit margin.

### Technical Analyst

- **Role**: Technical Analyst
- **Goal**: Analyze {ticker}'s historical price data to identify trends and patterns that can help predict future price movements.
- **Backstory**: You are an expert in technical indicators for stock prices, and use them to analyze the trend, momentum and volatility of stocks.

### Stock Advisor

- **Role**: Stock Advisor
- **Goal**: Provide investment recommendations to whether buy, sell, or hold {ticker} based on fundamental and technical analysis.
- **Backstory**: You are a world class stock picker and provide advice to clients based on fundamental and technical analysis.

## Tasks

### Fundamental Analysis

- **Description**: Analyze {ticker}'s fundamental data to evaluate the company's profitability and growth potential. Use the provided tool to analyze total revenue, net income, earnings per share, net profit margin, and possibly other key financial metrics. Use available Trailing Twelve Months (TTM) data in your analysis if necessary.
- **Expected Output**: The final answer should be a summary of the company's financial health and growth prospects based on the data available.
- **Agent**: fundamental_analyst

### Technical Analysis

- **Description**: Analyze {ticker}'s historical price data to predict future price movements. Use the provided tool to analyze price trends, momentum, and volatility. For momentum use the RSI indicator, and for volatility use the Bollinger Bands percentage indicator. Use the available historical price data in the tool to analyze the stock's price movements.
- **Expected Output**: The final answer should be a summary of the company's price trends and potential future price movements based on the data available.
- **Agent**: technical_analyst

### Stock Advisor Task

- **Description**: Analyze {ticker}'s fundamental and technical data to provide a recommendation on whether to buy, sell, or hold the stock. Use the provided input to analyze both fundamental and technical data for a comprehensive view of the stock's investment potential.
- **Expected Output**: The final answer should be a recommendation (buy, sell, or hold) based on the analysis of the company's profitability, historical fundamental data and technical indicators. The recommendation should be supported by the data available, and should be clear and concise. Highlight the latest fundamental and technical data that support your recommendation.
- **Agent**: stock_advisor

